In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os       
import sys
import glob 
import logging
import plotly.graph_objects as go
from tabulate import tabulate

In [2]:
# Configure the logger
logging.basicConfig(
    level=logging.INFO,  
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(), 
        logging.FileHandler("app.log") 
    ]
)

logger = logging.getLogger(__name__)

In [3]:
data = pd.read_parquet(r'D:\Ayush\Assignment\data\candles\BANKNIFTY\2024-01-10\46900PE.parquet.gz')
# print(tabulate(data, headers='keys', tablefmt='grid'))

In [4]:
class CandleConversion:
    def __init__(self):
        # self.df  = df
        self.new_data = None

    def time_interval(self,t):
         value = int(t[:-1])
         unit = t[-1]

         if unit=='m':
              return value
         if unit=='h':
              return value*60
         if unit=='d':
              return value*60*24
         else:
              raise ValueError(f"Invalid Time Interval")

    def process_data(self):
        pri=None
        prices = ['46900PE','47100CE','47200CE','47400CE','47400PE','47600CE','47800PE']
        
        # list(strike_price)
        input_data= input('Enter the input data path') or None
        time = input('Enter the time interval for the candle combinator(1m,3m,30m)') or '5m'
        strike_price = input('Which strike price would you want to convert(If you wish to convert all of them enter "all")') or 'all'
        path = input('Please provide the path where you want to save the files') or None
        t= self.time_interval(time)

        if input_data is not None and os.path.exists(input_data):
             df= pd.read_parquet((input_data))
             specific_date = '2024-01-10'
             df = df[df['date'].dt.date == pd.to_datetime(specific_date).date()]
             self.new_data= self.combination(df,t)
             if path is None:
                    self.new_data.to_csv(f'{time}_{pri}.csv')
                    logger.info("Output will be saved in the current directory")
             else:
                    name= f'{time}_{pri}.csv'
                    path = os.path.join(path,name)
                    logger.info(f"Output is saving in the path-> {path}")
                    self.new_data.to_csv(path, index = False)
          
            


        elif strike_price== 'all':

                for i in range(len(prices)):
                    pri = prices[i]
                    data_path = f'D:/Ayush/Assignment/data/candles/BANKNIFTY/2024-01-10/{pri}.parquet.gz'
                    df= pd.read_parquet((data_path))
                    specific_date = '2024-01-10'
                    df = df[df['date'].dt.date == pd.to_datetime(specific_date).date()]
                    self.new_data= self.combination(df,t)
                    if path is None:
                        self.new_data.to_csv(f'{time}_{pri}.csv')
                        logger.info("Output will be saved in the current directory")
                    else:
                        name= f'{time}_{pri}.csv'
                        path = os.path.join(path,name)
                        logger.info(f"Output is saving in the path-> {path}")
                        self.new_data.to_csv(path, index = False)
        elif strike_price != 'all':
            if strike_price in prices:
               pri = strike_price
               data_path = f'D:/Ayush/Assignment/data/candles/BANKNIFTY/2024-01-10/{pri}.parquet.gz'
               df= pd.read_parquet((data_path))
               specific_date = '2024-01-10'
               df = df[df['date'].dt.date == pd.to_datetime(specific_date).date()]
               self.new_data= self.combination(df,t)
               if path is None:
                        file_name = f'{time}_{pri}.csv'
                        self.new_data.to_csv(file_name)
                        logger.info("Output will be saved in the current directory")
               else:
                        name= f'{time}_{pri}.csv'
                        path = os.path.join(path,name)

                        logger.info(f"Output is saving in the path-> {path}")
                        self.new_data.to_csv(path, index = False)
            else:
                 logger.warning(f'Data is not avaliable for {strike_price}')
             
        show = input('Do you wish to see the final data(Y or n)')
        if show=='Y':
             return self.get_data()
        elif show =='n':
             logger.info('Data saved Successfully')
             return None
        else:
             logger.warning('Invalid Command')

        
    def combination(self,df,t):
            logger.info("Candle Comination Started")
            columns = ['date','open','high','low','close']
            rows = []

            for i in range(0,len(df),t):
                if i+t-1<=len(df):
                    date = df.iloc[i]['date']
                    high = max(df.iloc[i:i+t]['high'])
                    open = df.iloc[i]['open']
                    close= df.iloc[i+t-1]['close']
                    low = min(df.iloc[i:i+t]['low'])

                rows.append([date,open,high,low,close])
            
            self.new_data = pd.DataFrame(rows,columns=columns)
            logger.info(f"Combination of Candles into{t} minute candle is complete and the new data has {len(self.new_data)} rows")
          
            print("Combination Complete!")
            return self.new_data

    def get_data(self):
         if self.new_data is None:
              logger.warning("Data is not combined")
              return None
         
         else:
              logger.info('Data combining')
              print(tabulate(self.new_data, headers='keys', tablefmt='grid'))
              return self.new_data
    

In [5]:
converter = CandleConversion()
Converted = converter.process_data()

2025-05-09 22:11:22,350 - INFO - Candle Comination Started
2025-05-09 22:11:22,384 - INFO - Combination of Candles into5 minute candle is complete and the new data has 75 rows
2025-05-09 22:11:22,392 - INFO - Output will be saved in the current directory
2025-05-09 22:11:22,399 - INFO - Candle Comination Started
2025-05-09 22:11:22,426 - INFO - Combination of Candles into5 minute candle is complete and the new data has 75 rows
2025-05-09 22:11:22,429 - INFO - Output will be saved in the current directory
2025-05-09 22:11:22,437 - INFO - Candle Comination Started
2025-05-09 22:11:22,463 - INFO - Combination of Candles into5 minute candle is complete and the new data has 75 rows
2025-05-09 22:11:22,465 - INFO - Output will be saved in the current directory
2025-05-09 22:11:22,475 - INFO - Candle Comination Started
2025-05-09 22:11:22,502 - INFO - Combination of Candles into5 minute candle is complete and the new data has 75 rows
2025-05-09 22:11:22,505 - INFO - Output will be saved in th

Combination Complete!
Combination Complete!
Combination Complete!
Combination Complete!
Combination Complete!
Combination Complete!


2025-05-09 22:11:22,618 - INFO - Combination of Candles into5 minute candle is complete and the new data has 75 rows
2025-05-09 22:11:22,621 - INFO - Output will be saved in the current directory


Combination Complete!


2025-05-09 22:11:23,956 - INFO - Data combining


+----+---------------------+--------+--------+--------+---------+
|    | date                |   open |   high |    low |   close |
+====+=====================+========+========+========+=========+
|  0 | 2024-01-10 09:15:00 | 650    | 741.85 | 637    |  701.5  |
+----+---------------------+--------+--------+--------+---------+
|  1 | 2024-01-10 09:20:00 | 701.5  | 738    | 689.15 |  714.05 |
+----+---------------------+--------+--------+--------+---------+
|  2 | 2024-01-10 09:25:00 | 714.05 | 785    | 706.5  |  784.3  |
+----+---------------------+--------+--------+--------+---------+
|  3 | 2024-01-10 09:30:00 | 784.3  | 794.4  | 584.65 |  587    |
+----+---------------------+--------+--------+--------+---------+
|  4 | 2024-01-10 09:35:00 | 587    | 679    | 562.3  |  603    |
+----+---------------------+--------+--------+--------+---------+
|  5 | 2024-01-10 09:40:00 | 603    | 623.6  | 525    |  525    |
+----+---------------------+--------+--------+--------+---------+
|  6 | 202

FIBBONACCI PIVOT POINT

In [6]:
class Fibbonacci_Pivots:
    def __init__(self,df):
        self.df = df
        self.new_data = None
    
    def pivot_calculation(self):
        for i in range(len(self.df)):
            pivot = (self.df.iloc[i]['high']+self.df.iloc[i]['low']+self.df.iloc[i]['close'])/3
            self.df.loc[i,'pivot'] = pivot
            self.df.loc[i,'R1'] = pivot +0.382*(self.df.iloc[i]['high']-self.df.iloc[i]['low'])
            self.df.loc[i,'R2'] = pivot +0.681*(self.df.iloc[i]['high']-self.df.iloc[i]['low'])
            self.df.loc[i,'R3'] = pivot +(self.df.iloc[i]['high']-self.df.iloc[i]['low'])
            self.df.loc[i,'S1'] = max(pivot -0.382*(self.df.iloc[i]['high']-self.df.iloc[i]['low']),0)
            self.df.loc[i,'S2'] = max(pivot -0.681*(self.df.iloc[i]['high']-self.df.iloc[i]['low']),0)
            self.df.loc[i,'S3'] = max(pivot -(self.df.iloc[i]['high']-self.df.iloc[i]['low']),0)

        return self.df
    
    def plot_pivots_points_and_levels(self,data):

        fig = go.Figure()

        fig.add_trace(go.Scatter(
        x=data['date'],
        y=data['close'],
        mode='lines',
        name='Closing Price',
        line=dict(color='blue', width=2)
        ))

        levels = ['pivot','R1','R2','R3','S1','S2','S3']
        colors = {
            'pivot': 'orange',
            'R1': 'green',
            'R2': 'green',
            'R3': 'green',
            'S1': 'red',
            'S2': 'red',
            'S3': 'red'
        }

        for level in levels:
            if level in data.columns:
                fig.add_trace(go.Scatter(
                    x=data['date'],
                    y=data[level],
                    mode='lines',
                    name=level,
                    line=dict(color=colors[level], dash='dot', width=1.5)
                ))


        fig.update_layout(
            title="Closing Prices and Levels",
            xaxis_title="Date",
            yaxis_title="Price",
            template="plotly_white",
            width=1000,
            height=600,
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1
            )
        )

        fig.show()

    def user(self):
        q1 = input('Do you want to add the Resistance and Support Levels in your dataframe(Y or n)')
        if q1== 'Y':
            logger.info('Converting dataframe')
            self.new_data = self.pivot_calculation()
            
            q2= input('Do you want to plot the levels along with the prices(Y or n)')
            if q2=='Y':
                logger.info('Plotting Levels')
                self.plot_pivots_points_and_levels(self.new_data)
            elif q2=='n':
                logger.info('Skipping Plot')
                pass
            else:
                logger.error('Invalid Input')
                return
            


            print(tabulate(self.new_data, headers='keys', tablefmt='grid'))
            
            return self.new_data
        elif q1=='n':
            return None
        else:
            logger.error('Invlaid Input')



    

In [7]:
plotter = Fibbonacci_Pivots(Converted)
Final_Data =plotter.user()

2025-05-09 22:11:25,178 - INFO - Converting dataframe
2025-05-09 22:11:26,514 - INFO - Plotting Levels


+----+---------------------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+
|    | date                |   open |   high |    low |   close |   pivot |      R1 |      R2 |      R3 |      S1 |      S2 |      S3 |
+====+=====================+========+========+========+=========+=========+=========+=========+=========+=========+=========+=========+
|  0 | 2024-01-10 09:15:00 | 650    | 741.85 | 637    |  701.5  | 693.45  | 733.503 | 764.853 | 798.3   | 653.397 | 622.047 | 588.6   |
+----+---------------------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+
|  1 | 2024-01-10 09:20:00 | 701.5  | 738    | 689.15 |  714.05 | 713.733 | 732.394 | 747     | 762.583 | 695.073 | 680.466 | 664.883 |
+----+---------------------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+
|  2 | 2024-01-10 09:25:00 | 714.05 | 785    | 7